In [8]:
# %load ReturnPrediction_LSTM.py
"""
Created on Sat Apr  1 13:32:13 2017

@author: sonaa
"""


import os
import time
import warnings
import numpy as np
import pandas as pd
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Convolution1D, MaxPooling1D, Flatten,  Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, Normalizer

Using TensorFlow backend.
C:\Users\user\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
np.random.seed(7)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
"""
import os
os.environ[‘TF_CPP_MIN_LOG_LEVEL‘]=‘1‘ # 这是默认的显示等级，显示所有信息

# 2级
import os
os.environ[‘TF_CPP_MIN_LOG_LEVEL‘]=‘2‘ # 只显示 warning 和 Error

# 3级
import os
os.environ[‘TF_CPP_MIN_LOG_LEVEL‘]=‘3‘ # 只显示 Error

"""
warnings.filterwarnings("ignore") 
"""
Insert an entry into the list of warnings filters (at the front).

    'action' -- one of "error", "ignore", "always", "default", "module",
                or "once"
    'message' -- a regex that the warning message must match
    'category' -- a class that the warning must be a subclass of
    'module' -- a regex that the module name must match
    'lineno' -- an integer line number, 0 matches all warnings
    'append' -- if true, append to the list of filters
"""
df = pd.read_hdf('DeepLearning.h5', 'Data_Gold')


for c in df.columns:
    df[c+'_ret'] = df[c].pct_change().fillna(0)

def create_dataset(dataset, look_back=1, columns = ['Gold']):
    dataX, dataY = [], []
    for i in range(len(dataset.index)):
        if i <= look_back:
            continue
        a = None
        for c in columns:
            b = dataset.loc[dataset.index[i-look_back:i], c].as_matrix()
            if a is None:
                a = b
            else:
                a = np.append(a,b)
        dataX.append(a)
        dataY.append(dataset.loc[dataset.index[i], columns].as_matrix())
    return np.array(dataX), np.array(dataY)

look_back = 12
sc = StandardScaler()
df.loc[:, 'Gold'] = sc.fit_transform(df.loc[:, 'Gold'])
sc1 = StandardScaler()
df.loc[:, 'Inflation'] = sc1.fit_transform(df.loc[:, 'Inflation'])
sc2 = StandardScaler()
df.loc[:, 'InterestRate'] = sc1.fit_transform(df.loc[:, 'InterestRate'])
sc2 = StandardScaler()
df.loc[:, 'DJI'] = sc1.fit_transform(df.loc[:, 'DJI'])

train_df = df.loc[df.index < pd.to_datetime('2016-01-01')]

timeseries = np.asarray(df.Gold)
timeseries = np.atleast_2d(timeseries)
if timeseries.shape[0] == 1:
        timeseries = timeseries.T
X = np.atleast_3d(np.array([timeseries[start:start + look_back] for start in range(0, timeseries.shape[0] - look_back)]))
y = timeseries[look_back:]

predictors = ['Gold']#, 'DJI','Inflation']#, 'InterestRate']
#TRAIN_SIZE = train_x.shape[0]
#EMB_SIZE = look_back
model = Sequential()
model.add(LSTM(input_shape = (1,), input_dim=1, output_dim=6, return_sequences=True))
model.add(LSTM(input_shape = (1,), input_dim=1, output_dim=6, return_sequences=False))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss="mse", optimizer="rmsprop")
model.fit(X, 
          y, 
          epochs=1000, 
          batch_size=80, verbose=1, shuffle=False)



df['Pred'] = df.loc[df.index[0], 'Gold']
for i in range(len(df.index)):
    if i <= look_back:
        continue
    a = None
    for c in predictors:
        b = df.loc[df.index[i-look_back:i], c].as_matrix()
        if a is None:
            a = b
        else:
            a = np.append(a,b)
        a = a
    y = model.predict(a.reshape(1,look_back*len(predictors),1))
    df.loc[df.index[i], 'Pred']=y[0][0]
df.to_hdf('DeepLearning.h5', 'Pred_LSTM')
df.loc[:, 'Gold'] = sc.inverse_transform(df.loc[:, 'Gold'])
df.loc[:, 'Pred'] = sc.inverse_transform(df.loc[:, 'Pred'])
plt.plot(df.Gold,'y')
plt.plot(df.Pred, 'g')

In [6]:
*lstm?

In [7]:
#ctrl shift p: commands list

In [11]:
?os.environ

In [14]:
print(warnings.filterwarnings.__doc__) 

Insert an entry into the list of warnings filters (at the front).

    'action' -- one of "error", "ignore", "always", "default", "module",
                or "once"
    'message' -- a regex that the warning message must match
    'category' -- a class that the warning must be a subclass of
    'module' -- a regex that the module name must match
    'lineno' -- an integer line number, 0 matches all warnings
    'append' -- if true, append to the list of filters
    
